In [ ]:
!pip install xarray
!pip install ecmwflibs
!pip install cfgrib
!pip install numpy
!pip install pandas

In [ ]:
from typing import List
txt = str("filter_by_keys={'typeOfLevel': 'meanSea'}\n    filter_by_keys={'typeOfLevel': 'hybrid'}\n    filter_by_keys={'typeOfLevel': 'atmosphere'}\n    filter_by_keys={'typeOfLevel': 'surface'}\n    filter_by_keys={'typeOfLevel': 'planetaryBoundaryLayer'}\n    filter_by_keys={'typeOfLevel': 'isobaricInPa'}\n    filter_by_keys={'typeOfLevel': 'isobaricInhPa'}\n    filter_by_keys={'typeOfLevel': 'heightAboveGround'}\n    filter_by_keys={'typeOfLevel': 'depthBelowLandLayer'}\n    filter_by_keys={'typeOfLevel': 'heightAboveSea'}\n    filter_by_keys={'typeOfLevel': 'atmosphereSingleLayer'}\n    filter_by_keys={'typeOfLevel': 'lowCloudLayer'}\n    filter_by_keys={'typeOfLevel': 'middleCloudLayer'}\n    filter_by_keys={'typeOfLevel': 'highCloudLayer'}\n    filter_by_keys={'typeOfLevel': 'cloudCeiling'}\n    filter_by_keys={'typeOfLevel': 'heightAboveGroundLayer'}\n    filter_by_keys={'typeOfLevel': 'tropopause'}\n    filter_by_keys={'typeOfLevel': 'maxWind'}\n    filter_by_keys={'typeOfLevel': 'isothermZero'}\n    filter_by_keys={'typeOfLevel': 'highestTroposphericFreezing'}\n    filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'}\n    filter_by_keys={'typeOfLevel': 'sigmaLayer'}\n    filter_by_keys={'typeOfLevel': 'sigma'}\n    filter_by_keys={'typeOfLevel': 'potentialVorticity'}")

def cls_txt(det: str):

    

    det = det.lower()
    det = det.replace("=", "")
    det = det.replace("{", "")
    det = det.replace("}", "")
    det = det.replace("\n", "")
    det = det.replace(":", "")
    det = det.replace("'", "")
    det = det.replace("  ", " ")
    det = det.replace("filter_by_keys", "")
    det = det.replace("typeoflevel", "")
    det = det.replace("  ", "")
    det = det.lstrip()
    det = det.rstrip()
    det = det
    
    det = det.replace(" ", ", ")
    return det

txt2 = cls_txt(txt)

perams = []
txt3 = txt2.split(',')  # Split txt2 outside the loop
for n in range(len(txt3)):
    txt3[n] = txt3[n].lstrip()  # Corrected: Assign back the stripped string
    perams.append(txt3[n])  # Corrected: Append the stripped string to perams

perams.remove('hybrid')
perams

In [ ]:
### GFS DATA CURATION AND PREPROCESSING ###
data_path = r"C:\Projs\COde\Earthquake\earthquake-prediction\data\GFS" # Change DATA PATH


def wrf_prep_fn(pats, key_params):
    """
    
    """
    for root, dirs, files in os.walk(pats):
        for f in files:
            try:
                if f.endswith('.grb2'):
                    for key_param in perams:
                        try:
                            f_pt = os.path.join(root, f)
                            ini_ds = open_dataset(f_pt, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface'}}, decode_cf=True)
                            var_nams = list(loc_data(ini_ds, 22.71, 77.41).data_vars)

                            init_df = pd.DataFrame()
                            vals = []

                            for var_name in var_nams:  # Loop through variable names
                                
                                data_value = loc_data(ini_ds, 22.71, 77.41)[var_name].values
                                print(f"{var_name}: {data_value}")
                                init_df[var_name] = ''  # Use variable name as column name

                                vals.append(data_value)

                            init_df.loc[len(init_df)] = vals
                            ini_ds.close()
                            return init_df
                        except Exception as e:
                            print(f"{e}: Error opening the dataset file, probably hte required file format is not present.")
            except FileNotFoundError as e:
                print(f"{e}: Gridded Dataset File Not found, consider checking he directory.")

            # return data_value
            

dpfs = wrf_prep_fn(data_path, txt2)